In [177]:
import pandas as pd
import numpy as np

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeClassifier

In [179]:
df = pd.read_csv('titanic.csv')

In [180]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [181]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

### Split

In [182]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [183]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S
...,...,...,...,...,...,...,...
71,3,male,21.0,0,0,7.8958,S
106,3,male,21.0,0,0,7.8208,Q
270,1,male,46.0,0,0,75.2417,C
348,2,male,24.0,0,0,13.5000,S


In [184]:
y_test

321    0
324    1
388    0
56     0
153    1
      ..
57     0
126    0
24     1
17     0
66     1
Name: Survived, Length: 84, dtype: int64

### Imputation- Fill none values

In [185]:
#imputation transformer
trf1 = ColumnTransformer([
    #here [2] is the index of column
    ('impute_age',SimpleImputer(),[2]),                                 
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],
    remainder='passthrough')

### Encoding- One-Hot Encoding

- sparse=False : to retur numpy array
- handle_unknown='ignore' : 
- drop='first' : to handle colinearity delete the first encoded column

In [186]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

### Scaling

In [187]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

### Feature Selection

- k=n : n is the number of best feature to select

In [188]:
trf4  = SelectKBest(score_func=chi2,k=8)

### Model

In [189]:
trf5 = DecisionTreeClassifier()

### Create Pipeline

In [190]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

### Train

In [191]:
pipe.fit(X_train.values,y_train.values)
#to desplay piplining need below "set_config" code

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fe2e2d729d0>)),
                ('trf5', DecisionTreeClassifier())])

## Extra- Pipeline Vs make_pipeline

- Pipeline requires naming of steps, make_pipeline does not.
- Same applies to ColumnTransformer Vs make_colunm_transformer

In [192]:
#Alternate Syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [193]:
#To display pipline structure
from sklearn import set_config
set_config(display='diagram')

## Explore the pipeline

In [194]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7fe2e2d729d0>),
 'trf5': DecisionTreeClassifier()}

In [195]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [196]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [197]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_
#it returns the what does the first impute has returned
#likewise you can explore more REF: 29th VIDEO

array([30.11576336])

### Predict

In [198]:
y_pred =  pipe.predict(X_test)

/home/woyce-1-3/anaconda3/envs/django/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/home/woyce-1-3/anaconda3/envs/django/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [199]:
y_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

### Accuracy Calc

In [200]:
from sklearn.metrics import accuracy_score

In [201]:
accuracy_score(y_test,y_pred)

0.5952380952380952

### Cross Validation using Pipeline

In [202]:
from sklearn.model_selection import cross_val_score

In [203]:
cross_val = cross_val_score(pipe,X_train,y_train, cv=5, scoring='accuracy')
cross_val

array([0.56716418, 0.62686567, 0.62686567, 0.64179104, 0.65151515])

In [204]:
cross_val.mean()

0.622840343735866

### GridSearch using Pipeline - hyper parameter tuning

In [205]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]     #'modelname__parameternametotune'
}

In [206]:
from sklearn.model_selection import GridSearchCV

In [207]:
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7fe2e2d729d0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [208]:
grid.best_score_

0.6258254183627318

In [209]:
grid.best_params_

{'trf5__max_depth': 1}

### Exporting the Pipeline - to use in the production

In [210]:
import pickle

In [211]:
pickle.dump(pipe,open('pipe.pk1','wb'))